# Домашнее задание № 10. Генерация текста

### Задание 1 (10 баллов).

В семинаре мы работали с датасетами инструкций alpaca и dolly. Они англоязычные. В домашке вам нужно создать аналогичный датасет на русском языке и обучить аналогичную модель на этом датасете.
В качестве итогового результата у вас должна получится модель, которая может связно отвечать на русскоязычные инструкции на русском языке. Приведите как минимум три разных примера. Правильность ответов не так важна, так как вы скорее всего будете использовать небольшие модели, но текст должен быть не рандомным.

Русскоязычный датасет инструкций должен быть больше 5 тысяч примеров. Он может быть основнован на alpaca/dolly (например, вы можете просто прогнать все через переводную модель, которая была на семинаре, или даже google translate). Или вы можете придумать способ создать аналогичный датасет каким-то другим способом (переделать открытые датасеты с помощью правил). Датасет может быть не уникальным, можно скооперироваться с одногруппниками и сделать один датасет на всех.

Вы можете попробовать дообучать любую небольшую decoder-only модель. Скорее всего лучше всего будут работать модели, изначально обученные на русском языке (rugpt например). Но возможно даже модели вроде opt можно будет дообучить на русскоязычных инструкциях.

Это задание гораздно менее определенное, по сравнению с предыдущими. Поэтому не стесняйтесь задавать дополнительные вопросы в чате или лично, если у вас возникнут трудности.


Оказывается кто-то уже все перевел, можно вот этот датасет использовать для домашки - https://huggingface.co/datasets/IlyaGusev/ru_turbo_alpaca

## Датасет ru_turbo_alpaca

In [ ]:
#!pip uninstall torch torchvision torchaudio torchtext

In [ ]:
!pip install torch==2.2.2 torchvision
!pip install transformers[torch] pandas tokenizers datasets xformers zstandard jsonlines

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
import copy
import logging
from dataclasses import dataclass, field
from typing import Optional, Dict, Sequence
import json
import torch
import transformers
from torch.utils.data import Dataset
from transformers import Trainer

In [ ]:
dataset = load_dataset("IlyaGusev/ru_turbo_alpaca")

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
        num_rows: 29822
    })
})

In [ ]:
dataset['train'][:3]

{'instruction': ['Опишите, как сделать горшок из глины.',
  'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
  'Напиши аналог данной пословицы на современный лад: "Не говори гоп, пока не перепрыгнешь".'],
 'input': ['', '', ''],
 'output': ['Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.',
  'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и укра

In [ ]:
dataset['train'][3]

{'instruction': 'Представьте, что вы занимаетесь дизайном главной страницы для нового сайта. Опишите, что вы хотели бы видеть на этой странице.',
 'input': '',
 'output': 'Я бы хотел, чтобы главная страница была простой, но информативной. На ней должны быть ясно выделены основные разделы и функции сайта, а также действительно привлекательный визуальный дизайн. Я бы использовал яркие цвета и простые шрифты, чтобы все было легко читаемо. Также я бы добавил крупные изображения и логотип компании, чтобы сделать страницу более запоминающейся.',
 'alternative_output': 'Как СПА, мной, ИИ, нет предпочтений и личных вкусов. Но как эксперт, я могу предложить следующее:\n\n1. Ясный и привлекательный заголовок, который бы кратко и точно передавал суть сайта. Например, "Лучший маркетплейс для покупки и продажи товаров".\n\n2. Навигационная панель, которая бы позволила посетителям легко перемещаться по сайту. Должно быть понятно, где можно найти продукты, услуги, информацию о компании, контакты, обр

In [ ]:
IGNORE_INDEX = -100
DEFAULT_PAD_TOKEN = "[PAD]"
DEFAULT_EOS_TOKEN = "</s>"
DEFAULT_BOS_TOKEN = "</s>"
DEFAULT_UNK_TOKEN = "</s>"

PROMPT_DICT = {
    "prompt_input": (
        "Ниже приведено описание задания, сопровождаемое входными данными, которые предоставляют дополнительный контекст. "
        "Напиши ответ, который соответствует запросу.\n\n"
        "### Описание задания:\n{instruction}\n\n### Входные данные:\n{input}\n\n### Ответ:"
    ),
    "prompt_no_input": (
        "Ниже приведено описание задания. "
        "Напиши ответ, который соответствует запросу.\n\n"
        "### Описание задания:\n{instruction}\n\n### Ответ:"
    ),
}


In [ ]:
# превращение токенов в индексы и паддинг/урезание последовательностей

def _tokenize_fn(strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
    """Tokenize a list of strings."""
    tokenized_list = [
        tokenizer(
            text,
            return_tensors="pt",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        for text in strings
    ]
    input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
    input_ids_lens = labels_lens = [
        tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
    ]
    return dict(
        input_ids=input_ids,
        labels=labels,
        input_ids_lens=input_ids_lens,
        labels_lens=labels_lens,
    )

In [ ]:
def preprocess(
    sources: Sequence[str],
    targets: Sequence[str],
    tokenizer: transformers.PreTrainedTokenizer,
) -> Dict:
    """Preprocess the data by tokenizing."""
    examples = [s + t for s, t in zip(sources, targets)]
    examples_tokenized, sources_tokenized = [_tokenize_fn(strings, tokenizer) for strings in (examples, sources)]
    input_ids = examples_tokenized["input_ids"]
    labels = copy.deepcopy(input_ids)
    for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
        label[:source_len] = IGNORE_INDEX
    return dict(input_ids=input_ids, labels=labels)

In [ ]:
# Далее это оборачивается к классы, которые предобрабатывают данные к формату huggingface.
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""

    def __init__(self, data_path: str, tokenizer: transformers.PreTrainedTokenizer):
        super(SupervisedDataset, self).__init__()
        logging.warning("Loading data...")
        list_data_dict = json.load(open(data_path))

        logging.warning("Formatting inputs...")
        prompt_input, prompt_no_input = PROMPT_DICT["prompt_input"], PROMPT_DICT["prompt_no_input"]
        sources = [
            prompt_input.format_map(example) if example.get("input", "") != "" else prompt_no_input.format_map(example)
            for example in list_data_dict
        ]
        targets = [f"{example['output']}{tokenizer.eos_token}" for example in list_data_dict]

        logging.warning("Tokenizing inputs... This may take some time...")
        data_dict = preprocess(sources, targets, tokenizer)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])


@dataclass
class DataCollatorForSupervisedDataset(object):
    """Collate examples for supervised fine-tuning."""

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value=IGNORE_INDEX)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

In [ ]:

json_data = []

for example in dataset['train']:
    json_example = {
        'instruction': example['instruction'],
        'input': example['input'],
        'output': example['output']
    }
    json_data.append(json_example)

with open('dataset.json', 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)


In [ ]:
data_json = json.load(open('dataset.json'))

In [ ]:
data_json[:3]

[{'instruction': 'Опишите, как сделать горшок из глины.',
  'input': '',
  'output': 'Для изготовления горшка из глины сначала необходимо взять глину и размешать ее водой до состояния пластилина. Затем нужно сформировать глиняный комок нужной формы и тщательно откачать влагу из глины, чтобы она стала твердой. Горшок нужно обжечь в духовке при высокой температуре, чтобы он стал прочным и непроницаемым.'},
 {'instruction': 'Опиши процесс изготовления торта. Укажи ингредиенты, необходимые для этого.',
  'input': '',
  'output': 'Для изготовления торта необходимы следующие ингредиенты: мука, сахар, яйца, молоко, сливочное масло, разрыхлитель, ванилин. Сначала нужно смешать муку с сахаром и добавить яйца, молоко, разрыхлитель и ваниль. Затем тесто надо хорошо перемешать и разделить на несколько частей. Каждую часть нужно запекать в духовке до готовности. После этого торт надо собрать, сложив слои пирога вместе с начинкой и украшениями.'},
 {'instruction': 'Напиши аналог данной пословицы на 

## Модель rugpt3small_based_on_gpt2

In [ ]:
# загружаем модель
# https://huggingface.co/ai-forever/rugpt3small_based_on_gpt2

model_name = "ai-forever/rugpt3small_based_on_gpt2"
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name,
        max_length=512,
        cache_dir="huggingface_cache",
    )

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/551M [00:00<?, ?B/s]

In [ ]:
# Загружаем токенизатор
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_name,
    cache_dir="huggingface_cache",
    model_max_length=512,
    padding_side="right",
    use_fast=False,
)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

In [ ]:
# Токенизируем данные
train_dataset = SupervisedDataset(tokenizer=tokenizer, data_path='dataset.json')
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

In [ ]:
# Задаем параметры обуечения
from transformers import TrainingArguments

train_args = TrainingArguments(
    learning_rate=1e-5,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    evaluation_strategy='no',
    weight_decay=0.,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    save_strategy='no',
    logging_steps=1000,
    output_dir="rugpt3small-ru_turbo_alpaca-ft"
)

In [ ]:
# Обучаем
trainer = Trainer(model=model,
                 tokenizer=tokenizer,
                 args=train_args,
                 train_dataset=train_dataset,
                 eval_dataset=None,
                 data_collator=data_collator)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
# Сохранение модели
trainer.save_model('rugpt3small-ru_turbo_alpaca-ft')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512}


In [ ]:
MODEL_NAME = 'rugpt3small-ru_turbo_alpaca-ft'

In [ ]:
import shutil
import os
from google.colab import files

folder_path = '/content/rugpt3small-ru_turbo_alpaca-ft'
zip_path = '/content/rugpt3small-ru_turbo_alpaca-ft.zip'
shutil.make_archive(zip_path, 'zip', folder_path)

'/content/rugpt3small-ru_turbo_alpaca-ft.zip.zip'

## Проверка работы дообученной модели

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, model_max_length=512, max_length=512)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, max_length=512)

In [ ]:
def predict_for_instruction(instruction, text, model):
    text = text.replace('\n', ' ')
    prompt = ("Ниже приведено описание задания, сопровождаемое входными данными, которые предоставляют дополнительный контекст. "
              "Напиши ответ, который соответствует запросу.\n\n"
              f"### Описание задания:\n{instruction}\n\n### Входные данные:\n{text}\n\n### Ответ:")

    inputs = tokenizer([prompt],
                        return_tensors="pt", padding=True)

    output_sequences = model.generate(
        # this parameters are also important but you can read about them in the docs and just try changing them
        num_beams=1,
#         temperature=0.4,
#         max_length=100,
        max_new_tokens=20,
#         no_repeat_ngram_size=3,
    #     repetition_penalty= 5.0,
    #     length_penalty=0.01,
    #     early_stopping=True,
    #     do_sample=True,
    #     top_k=30,
    #     top_p=0.8,
        early_stopping=True,
    #     num_return_sequences=3,
        num_return_sequences= 1,
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        do_sample=False,  # disable sampling to test if batching affects output
    )
    summaries = tokenizer.batch_decode(output_sequences[:,len(inputs[0]):], skip_special_tokens=True)
    return summaries[0]

In [ ]:
text = """
Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов, что привело к длительным отменам по крайней мере на двух линиях.
Во вторник днем все поезда были остановлены на оживленной линии между южными городами Ден Босх и Бокстел после того, как животные зарылись в дамбу, несущую рельсы. Национальная железнодорожная компания сообщила, что линия не будет работать как минимум неделю.
Подкоп означает, что "рельсы могут проседать, и тогда безопасность движения поездов больше не гарантируется", - говорится в заявлении ProRail, компании, обслуживающей железнодорожную сеть Нидерландов.
Ранее в этом месяце барсуки также зарылись под рельсы в районе северной деревни Молкверум в провинции Фрисландия, что вывело линию из эксплуатации до следующего месяца, пока рабочие будут добиваться разрешения на перемещение животных.
Барсуки являются охраняемыми животными в Нидерландах, поэтому железнодорожники должны получить разрешение на их перемещение или нарушение их среды обитания, прежде чем начать ремонт.
"""

In [ ]:
instruction = "Перескажи этот текст."
predict_for_instruction(instruction, text, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:453: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'Барсуки, зарывшиеся под железнодорожными путями, остановили движение поездов на севере и юге Нидерландов,'

In [ ]:
text = """
"""

In [ ]:
instruction = "Расскажи анекдот"
predict_for_instruction(instruction, text, model)

'В магазине продавец спрашивает покупателя: "Сколько стоит килограмм мяса?" - "Двадцать пять рублей". -'

In [ ]:
instruction = "Как правильно варить суп?"
predict_for_instruction(instruction, text, model)

'Для приготовления супов нужно сварить бульон, добавить нарезанный лук, морковь, картофель, картофель'

In [ ]:
instruction = "Что поделать на каникулах?"
predict_for_instruction(instruction, text, model)

'На каникулах я бы хотел провести время с друзьями и семьей. Мы бы посетили музеи, теат'

In [ ]:
instruction = "Если бы ты стал человеком, что бы ты сделал первым делом?"
predict_for_instruction(instruction, text, model)

'Я бы начал с того, чтобы начать с того, чтобы начать жить в гармонии с собой и окружающим'

In [ ]:
instruction = "Чем ты любишь заниматься?"
predict_for_instruction(instruction, text, model)

'Я люблю заниматься спортом, особенно бегом. Я люблю бегать по утрам, чтобы улучшить свое здоровье и повысить'

In [ ]:
instruction = "Как правильно сочинять музыку?"
predict_for_instruction(instruction, text, model)

'Чтобы правильно сочинять музыку, нужно сначала определить жанр музыки и выбрать соответствующий жанр. Затем нужно выбрать исполнителя'

In [ ]:
instruction = "В чем смысл жизни?"
predict_for_instruction(instruction, text, model)

'Жизнь - это непрерывный процесс, который требует постоянного развития и развития. Она включает в себя множество этапов'